In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

Reading the data from excel

In [ ]:
train = pd.read_csv('/kaggle/input/mobile-price-classification/train.csv')
test = pd.read_csv('/kaggle/input/mobile-price-classification/test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
train.describe()

Check any duplicate in dataset

In [ ]:
doplic = train.duplicated()
doplic.head()

In [ ]:
sum(doplic)

Categorical columns: blue(Bluetooth), dual_sim, four_g,three_g, touch_screen, wifi,price_range

Let's seprate the numerical and categorical data

In [ ]:
numeric_data = train.drop(['blue','dual_sim','four_g','three_g','touch_screen','wifi','price_range'],axis=1) #axis=1 column wise
numeric_data.head()

In [ ]:
categorical_data = train[['blue','dual_sim','four_g','three_g','touch_screen','wifi','price_range']]
categorical_data.head()

In [ ]:
for col in numeric_data.columns:
    plt.subplots()
    sns.boxplot(numeric_data[col],orient='v')

Let's calculate the z-score for the data 

In [ ]:
scaler = StandardScaler()
scaler_array = scaler.fit_transform(numeric_data)

In [ ]:
scaled_data = pd.DataFrame(scaler_array,columns=numeric_data.columns)
scaled_data.head()

In [ ]:
scaled_data.describe()

In [ ]:
plt.subplots(figsize=(10,8))
bp = sns.boxplot(data = scaled_data )
bp.set_xticklabels(bp.get_xticklabels(),rotation=90)

In [ ]:
Q1 = scaled_data.quantile(0.25)
Q3 = scaled_data.quantile(0.75)

IQR = Q3 - Q1
print(IQR)

In [ ]:
outlier_removed_data = scaled_data[~((scaled_data < (Q1 - 1.5 * IQR)) | (scaled_data > (Q3 + 1.5 * IQR))).any(axis=1)]
outlier_removed_data.shape

In [ ]:
plt.subplots(figsize=(10,8))
bp = sns.boxplot(data = outlier_removed_data )
bp.set_xticklabels(bp.get_xticklabels(),rotation=90)

**Model Training**

In [ ]:
scaled_data = scaled_data.reset_index()
categorical_data =  categorical_data.reset_index()

In [ ]:

final_df = pd.concat([scaled_data,categorical_data],axis=1)
final_df = final_df.drop(['index'],axis=1)
final_df.head()

In [ ]:
final_df.info()

In [ ]:
X = final_df.drop('price_range',axis=1)
Y = final_df['price_range']
X.head()

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.20,random_state=101)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logistic_regression = LogisticRegression(solver='lbfgs',multi_class='multinomial',max_iter=10000)

In [ ]:
logistic_regression.fit(X_train,Y_train)

In [ ]:
logistic_regression.score(X_test,Y_test)